# **Sesión 2:** Calibración de Cámara 📷⚙️

## 0. Preparación

In [4]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import os
import imageio
import cv2
import copy
import glob

OpenCV should be 4.8.0.76 Current version: 4.8.0


## **Apartado A: Calibración de cámara** (derecha e izquierda)

En este apartado se realiza la calibración de dos cámaras de un sistema estereoscópico. Para ello se trabajará con las imágenes de las carpetas ``left`` y ``right``. En primer lugar se trabajará con la carpeta ``left``. Posteriormente, deberá repetir el proceso con las imágenes en la carpeta ``right``. Ambas carpetas contienen imágenes con las que se calibrarán las cámaras. En ellas aparece el patrón de calibración en diferentes posiciones y orientaciones. Estas imágenes serán los datos de entrada.

Los pasos que deberá seguir para calibrar una cámara son:

1. Defina y ejecute el método para cargar imágenes ``load_images()``.
2. Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.
3. Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``.
4. Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)`` que proporcione las coordenadas 3D de las esquinas del patrón. El sistema de referencia utilizado deberá estar anclado al propio patrón.
5. Utilice ``cv2.calibrateCamera`` para obtener los parámetros de calibración para la cámara izquierda.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``.

In [20]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [21]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = []
for i in range(12):
    imgs_path.append(f"../calibration/{i}.jpg")
imgs = load_images(imgs_path)

C:\Users\javie\AppData\Local\Temp\ipykernel_30100\283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [22]:
# TODO Find corners with cv2.findChessboardCorners()
pattern_size = (9, 7)

corners = []
imgs_gray = []

for img in imgs:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgs_gray.append(gray)

    ret, corners_img = cv2.findChessboardCorners(gray, pattern_size, None)
    corners.append([ret, corners_img])


In [23]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.

corners_refined = [cv2.cornerSubPix(i, cor[1], (9, 7), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [24]:
imgs_copy = copy.deepcopy(imgs)

In [25]:
# TODO Show images and save when needed

def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def write_image(output_folder: str, img_name: str, img: np.array):
    img_path = os.path.join(output_folder, img_name)
    cv2.imwrite(img_path, img)

In [26]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
for img, ret, corners in zip(imgs_copy, [c[0] for c in corners], corners_refined):
    if ret:
        cv2.drawChessboardCorners(img, (9, 7), corners, ret)
        show_image(img, "Corner detection")

cv2.destroyAllWindows()


### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [27]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    cols, rows = chessboard_shape
    chessboard_points = np.zeros((rows * cols, 3), np.float32)
    chessboard_points[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2) * (dx, dy)
    return chessboard_points


In [28]:
# TODO You need the points for every image, not just one (consider a list comprehension)
chessboard_point = get_chessboard_points((9, 7), 20, 20)
chessboard_points = [chessboard_point for cor in corners_copy if cor[0]]


### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [29]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners_copy if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [30]:
# TODO
h, w = imgs_gray[0].shape[:2]

rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points, valid_corners,(w, h), None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [31]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[935.20296129   0.         656.08451705]
 [  0.         936.54621252 325.31221018]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[-2.00374519e-01  6.56376021e-01  6.89161711e-04  5.95954598e-03
  -1.25401050e+00]]
Root mean squared reprojection error:
 0.4288905001382016
